In [1]:
import nltk

# Commented out IPython magic to ensure Python compatibility.
# %cd /content/ease/ease

## IMPORTS ##
from essay_set import EssaySet

from feature_extractor import FeatureExtractor
from predictor_set import PredictorSet
from predictor_extractor import PredictorExtractor
from sklearn.svm import SVR
import pickle
import pandas as pd
import csv

import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import learning_curve,GridSearchCV
#from sklearn import svm, grid_search

import nltk
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')

# The following 3 functions have been taken from Ben Hamner's github repository
# https://github.com/benhamner/Metrics
def Cmatrix(rater_a, rater_b, min_rating=None, max_rating=None):
    """
    Returns the confusion matrix between rater's ratings
    """
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(rater_a + rater_b)
    if max_rating is None:
        max_rating = max(rater_a + rater_b)
    num_ratings = int(max_rating - min_rating + 1)
    conf_mat = [[0 for i in range(num_ratings)]
                for j in range(num_ratings)]
    for a, b in zip(rater_a, rater_b):
        conf_mat[a - min_rating][b - min_rating] += 1
    return conf_mat


def histogram(ratings, min_rating=None, max_rating=None):
    """
    Returns the counts of each type of rating that a rater made
    """
    if min_rating is None:
        min_rating = min(ratings)
    if max_rating is None:
        max_rating = max(ratings)
    num_ratings = int(max_rating - min_rating + 1)
    hist_ratings = [0 for x in range(num_ratings)]
    for r in ratings:
        hist_ratings[r - min_rating] += 1
    return hist_ratings


def quadratic_weighted_kappa(y, y_pred):
    """
    Calculates the quadratic weighted kappa
    axquadratic_weighted_kappa calculates the quadratic weighted kappa
    value, which is a measure of inter-rater agreement between two raters
    that provide discrete numeric ratings.  Potential values range from -1
    (representing complete disagreement) to 1 (representing complete
    agreement).  A kappa value of 0 is expected if all agreement is due to
    chance.
    quadratic_weighted_kappa(rater_a, rater_b), where rater_a and rater_b
    each correspond to a list of integer ratings.  These lists must have the
    same length.
    The ratings should be integers, and it is assumed that they contain
    the complete range of possible ratings.
    quadratic_weighted_kappa(X, min_rating, max_rating), where min_rating
    is the minimum possible rating, and max_rating is the maximum possible
    rating
    """
    rater_a = y
    rater_b = y_pred
    min_rating=None
    max_rating=None
    rater_a = np.array(rater_a, dtype=int)
    rater_b = np.array(rater_b, dtype=int)
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(min(rater_a), min(rater_b))
    if max_rating is None:
        max_rating = max(max(rater_a), max(rater_b))
    conf_mat = Cmatrix(rater_a, rater_b,
                                min_rating, max_rating)
    num_ratings = len(conf_mat)
    num_scored_items = float(len(rater_a))

    hist_rater_a = histogram(rater_a, min_rating, max_rating)
    hist_rater_b = histogram(rater_b, min_rating, max_rating)

    numerator = 0.0
    denominator = 0.0

    for i in range(num_ratings):
        for j in range(num_ratings):
            expected_count = (hist_rater_a[i] * hist_rater_b[j]
                              / num_scored_items)
            d = pow(i - j, 2.0) / pow(num_ratings - 1, 2.0)
            numerator += d * conf_mat[i][j] / num_scored_items
            denominator += d * expected_count / num_scored_items

    return (1.0 - numerator / denominator)

In [2]:
train_set = pd.read_csv('/home/mehar/github/ease/ease/train_adv/mixture_all_1_train_valid_reduce.csv', sep=',')
x = train_set.to_numpy()
tester = x.tolist()
print(len(tester))
essaylist = []
scorelist = []
for i in range(0, len(tester)):
    z = tester[i]
#     print(z[1])
#     y = z[0].split(', ', 1)
    #print(y)
    scorelist.append(float(z[0]))
    essaylist.append(z[1])

train = EssaySet()
print("Done1")
for i in range(0, len(essaylist)):
    train.add_essay(essaylist[i], scorelist[i])

print("Done2")
features=FeatureExtractor()
features.initialize_dictionaries(train)
X = features.gen_feats(train)
print("features train", X)
print("Done3")


1426
Done1
Done2
features train [[9.780e+02 2.050e+02 1.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]
 [2.134e+03 4.500e+02 1.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]
 [1.667e+03 3.690e+02 2.000e+00 ... 0.000e+00 0.000e+00 0.000e+00]
 ...
 [1.686e+03 3.470e+02 2.000e+01 ... 2.000e+00 1.000e+00 0.000e+00]
 [1.875e+03 3.900e+02 3.000e+00 ... 1.000e+00 0.000e+00 0.000e+00]
 [1.795e+03 3.700e+02 1.700e+01 ... 1.000e+00 0.000e+00 0.000e+00]]
Done3


In [3]:
test_score_set = pd.read_csv('scores/test1.csv', encoding = 'utf=8')
x_2 = test_score_set.to_numpy()
tester_2 = x_2.tolist()

test_scorelist = []

for i in range(0, len(tester_2)):
    z = tester_2[i]
    test_scorelist.append(float(z[0]))
print(len(test_scorelist))

357


In [4]:
# import os 
# import statistics
# file_path = "AES_testcases/prompt1/"
# files = os.listdir(file_path)
# files = sorted(files)
# files.pop(0)

# print(len(files), files)

In [21]:
# import os 
# import statistics
# file_path = "AES_testcases/prompt1/"
# files = os.listdir(file_path)
# files = sorted(files)
# for file in files:
#     print(file)
# # print(files)
import csv
x = 1
if x ==1:
    test_score_set = pd.read_csv('scores/test1.csv', encoding = 'utf=8')
    x_2 = test_score_set.to_numpy()
    tester_2 = x_2.tolist()

    test_scorelist = []

    for i in range(0, len(tester_2)):
        z = tester_2[i]
#         print("###")
#         print(z[0])
        test_scorelist.append(float(z[0]))
    print(len(test_scorelist))
    
#     print(testcase)
    test_set = pd.read_csv('AES_testcases/scores/test1.csv', encoding="latin1")
#     print(test_set.head(5))
    x = test_set.to_numpy()
    tester = x.tolist()
    test_essaylist = []

    for i in range(0, len(tester)):
        z = tester[i]
#         print(z)
#         y = z[0].split(', ', 1)
#         print(z)
        test_essaylist.append(z)
#     print(test_essaylist[0])
#     print("\n", test_essaylist[1])
    test = EssaySet(essaytype="test")
#     if len(test_essaylist) > len(test_scorelist):
#         test_essaylist = test_essaylist[:len(test_scorelist)]
#     else:
#         test_scorelist = test_scorelist[:len(test_essaylist)]
    print(len(test_essaylist), len(test_scorelist))
    
    
    for i in range(0, len(test_essaylist)):
        test.add_essay(test_essaylist[i], test_scorelist[i])
    Y = features.gen_feats(test)
#     print("features test", Y)
#     print("Done4")

    ## SCALING
    scaled_train = []
    for i in range(0, len(scorelist)):
        scaled_train.append(float((np.clip((scorelist[i]), a_min=2, a_max=12)/12)))

#     print("start training and prediciton")

    # print(scaled_train)
    from sklearn.svm import SVR

    # Values for prompt1
    Cs = [1]
    gammas = [0.00001]
    param_grid = {'C': Cs, 'gamma' : gammas}
    clf = GridSearchCV(SVR(kernel='rbf'), param_grid, cv =5)
    clf.fit(X, scaled_train)
#     print(clf.best_params_)

    final = clf.predict(Y)

    finals = (np.clip(final,a_min=0,a_max=1)*12)
    finals_list = finals.tolist()
    
    print(quadratic_weighted_kappa(test_scorelist,finals))
    
#     df_pred = pd.DataFrame(finals_list)
#     df_org = pd.DataFrame(test_scorelist)
#     print("pred", df_pred.head(5))
#     print("org", df_org.head(5))
    
#     df_pred.to_csv('predicted_scores/prompt1/%s'%(testcase), index=False, header=None)
        
#     mean_diff = statistics.mean(finals_list)
# #     print("Mean", statistics.mean(diff_list))
#     print(""+"{:.3f}".format(mean_diff))

#     neg_list =[]
#     for i in range(0, len(finals_list)):
#     #     print(finals_list[i], test_scorelist[i])
#         diff = test_scorelist[i] - finals_list[i]
#         if diff < 0:
#             neg_list.append(diff)
#     print(len(neg_list))
    
#     print("", statistics.mean(diff_list))
#     mean1 = statistics.mean(diff_list)
#     print(""+"{:.3f}".format(mean1))
#     sdev = statistics.stdev(finals_list)
#     print(""+"{:.3f}".format(sdev))
    
#     print(len(finals_list), len(test_scorelist))
#     diff_list =[]
#     for i in range(0, len(finals_list)):
#     #     print(finals_list[i], test_scorelist[i])
#         diff = test_scorelist[i] - finals_list[i]
#         diff_list.append(diff)
#     mean_diff = statistics.mean(diff_list)
#     print(""+"{:.3f}".format(mean_diff))
    
# #     print("Mean", statistics.mean(diff_list))
#     print(""+"{:.3f}".format(mean_diff))
#         # print(len(finals_list), len(test_scorelist))



356
356 356
0.0


In [22]:
finals

array([3.63919749, 3.63919749, 3.63919749, 3.63919749, 3.63919749,
       3.63919749, 3.63919749, 3.63919749, 3.63919749, 3.63919749,
       3.63919749, 3.63919749, 3.63919749, 3.63919749, 3.63919749,
       3.63919749, 3.63919749, 3.63919749, 3.63919749, 3.63919749,
       3.63919749, 3.63919749, 3.63919749, 3.63919749, 3.63919749,
       3.63919749, 3.63919749, 3.63919749, 3.63919749, 3.63919749,
       3.63919749, 3.63919749, 3.63919749, 3.63919749, 3.63919749,
       3.63919749, 3.63919749, 3.63919749, 3.63919749, 3.63919749,
       3.63919749, 3.63919749, 3.63919749, 3.63919749, 3.63919749,
       3.63919749, 3.63919749, 3.63919749, 3.63919749, 3.63919749,
       3.63919749, 3.63919749, 3.63919749, 3.63919749, 3.63919749,
       3.63919749, 3.63919749, 3.63919749, 3.63919749, 3.63919749,
       3.63919749, 3.63919749, 3.63919749, 3.63919749, 3.63919749,
       3.63919749, 3.63919749, 3.63919749, 3.63919749, 3.63919749,
       3.63919749, 3.63919749, 3.63919749, 3.63919749, 3.63919

In [30]:
df.to_csv('predicted_scores/test1_pred.csv', index=False, header=None)